# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

In [2]:
# pandas print options

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)
pd.set_option('display.colheader_justify', 'center')
pd.set_option('display.precision', 7)

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
# Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS orangeshark
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('orangeshark')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### Key references

* 0  - artist
* 1  - firstName of user
* 2  - gender of user
* 3  - item number in session
* 4  - last name of user
* 5  - length of the song
* 6  - level (paid or free song)
* 7  - location of the user
* 8  - sessionId
* 9  - song title
* 10 - userId

In [9]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

#
# SELECT artist, song_title, song_length FROM music_app_history1 WHERE session_id = 338 AND item_in_session = 4
#

### Query# 1 Notes

- Query# 1 requires the artist, song and song length data based on sessionId, itemInSession.

- So, chose ***sessionId and itemInSession*** as the PRIMARY KEY 

In [10]:
# Create TABLE for query# 1

query1_create = "CREATE TABLE IF NOT EXISTS songs_played_in_session "
query1_create = query1_create + "(session_id INT, item_in_session INT, artist TEXT \
                            , song_title TEXT, song_length FLOAT \
                            , PRIMARY KEY(session_id, item_in_session)); "

try:
    session.execute(query1_create)
except Exception as e:
    print(e)
    

In [11]:
# INSERT data into table for query# 1

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        try:
            query1_insert = "INSERT INTO songs_played_in_session (session_id, \
                            item_in_session, artist, song_title, song_length) " 
            query1_insert = query1_insert + " VALUES (%s, %s, %s, %s, %s); "
            session.execute(query1_insert, (int(line[8]), int(line[3]), line[0],
                                            line[9], float(line[5])))
        except Exception as e:
            print(e)

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
query = "select count(*) as count from songs_played_in_session "
try:
    rows = list(session.execute(query))
except Exception as e:
    print(e)
    
print(rows)

[Row(count=6820)]


## Query# 1 result


In [13]:
query1_select = "SELECT artist, song_title, song_length \
                 FROM songs_played_in_session \
                 WHERE session_id = 338 AND item_in_session = 4 ; "

try:
    rows = session.execute(query1_select)
except Exception as e:
    print(e)

print(" Query# 1 Results - song played in the session_id with 338 and item_in_session with 4 ")
print(" ------------------------------------------------------------------------------------ ")
print(pd.DataFrame(list(rows)))

 Query# 1 Results - song played in the session_id with 338 and item_in_session with 4 
 ------------------------------------------------------------------------------------ 
    artist              song_title             song_length
0  Faithless  Music Matters (Mark Knight Dub)  495.307312 


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [14]:
## Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182

# SELECT artist, song_title, user_first_name, user_last_name FROM music_app_history1 
# WHERE userid = 10 AND sessionid = 182 ORDER BY item_in_session ; 


### Query# 2 Notes

- Query# 2 requires the artist, song, user data based on userid, sessionId, and sorted by itemInSession.

- So, chose userid, sessionId as PRIMARY KEY, and itemInSession as clustering column


In [15]:
# Create TABLE for query# 2

query2_create = "CREATE TABLE IF NOT EXISTS songs_played_by_user "
query2_create = query2_create + "(user_id INT, session_id INT, item_in_session INT \
    , artist TEXT, song_title TEXT, user_first_name TEXT, user_last_name TEXT \
    , PRIMARY KEY((user_id, session_id), item_in_session)); "

try:
    session.execute(query2_create)
except Exception as e:
    print(e)
    

In [16]:
# INSERT data into table for query# 2

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query2_insert = "INSERT INTO songs_played_by_user (user_id, session_id, item_in_session, \
                        artist, song_title, user_first_name, user_last_name) "
        query2_insert = query2_insert + "VALUES (%s, %s, %s, %s, %s, %s, %s) ; "
        try:
            session.execute(query2_insert, (int(line[10]), int(line[8]), int(line[3]), line[0],
                                        line[9], line[1], line[4] ) )
        except Exception as e:
            print(e)

In [17]:
# Query# 2 result

query2_select = "SELECT artist, song_title, user_first_name, user_last_name \
                FROM songs_played_by_user \
                WHERE user_id = 10 AND session_id = 182; "

try:
    rows = session.execute(query2_select)
except Exception as e:
    print(e)
    
df_users_songs = pd.DataFrame(list(rows))

print(" Query# 2 Result - songs played by user_id with 10 and in session_id 182 ")
print(" ----------------------------------------------------------------------- ")
print(df_users_songs)

#for row in rows:
#    print("Artist: " + row.artist + ", Title: " + row.song_title +
#          ", User: " + row.user_first_name + ' ' + row.user_last_name)
    

 Query# 2 Result - songs played by user_id with 10 and in session_id 182 
 ----------------------------------------------------------------------- 
        artist                           song_title                     user_first_name user_last_name
0   Down To The Bone                                 Keep On Keepin' On      Sylvie          Cruz     
1       Three Drives                                        Greece 2000      Sylvie          Cruz     
2  Sebastien Tellier                                          Kilometer      Sylvie          Cruz     
3      Lonnie Gordon  Catch You Baby (Steve Pitron & Max Sanna Radio...      Sylvie          Cruz     


In [18]:
## Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# SELECT user_first_name, user_last_name AS user FROM music_app_history2 
# WHERE song_title = 'All Hands Against His Own'; "
                    

### Query# 3 Notes

- Query# 3 requires user data based on song_title.

- So, chose song_title as PRIMARY KEY with userid as clustering column

In [19]:
# Create TABLE for query# 3

query3_create = "CREATE TABLE IF NOT EXISTS song_listeners "
query3_create = query3_create + "(song_title TEXT, user_id INT, user_first_name TEXT, \
    user_last_name TEXT, PRIMARY KEY((song_title), user_id)); "

try:
    session.execute(query3_create)
except Exception as e:
    print(e)
    

In [20]:
# INSERT data into table for query# 3

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) 
    for line in csvreader:
        query3_insert = "INSERT INTO song_listeners(song_title, user_id, user_first_name, user_last_name) "
        query3_insert = query3_insert + " VALUES (%s, %s, %s, %s); "  
        try:
            session.execute(query3_insert, (line[9], int(line[10]), line[1], line[4]))
        except Exception as e:
            print(e)

In [21]:
# Query# 3 result

query3_select = "select user_first_name, user_last_name \
                FROM song_listeners \
                WHERE song_title = 'All Hands Against His Own' ; "
try:
    rows = session.execute(query3_select)
except Exception as e:
    print(e)

print(" Query# 3 Results - Users listened to 'All Hands Against His Own' song ")
print(" --------------------------------------------------------------------- ")
print(pd.DataFrame(list(rows)))

#for row in rows:
#    print (row.user_first_name + ' ' + row.user_last_name)

 Query# 3 Results - Users listened to 'All Hands Against His Own' song 
 --------------------------------------------------------------------- 
  user_first_name user_last_name
0    Jacqueline         Lynch   
1         Tegan        Levine   
2          Sara       Johnson   


### Drop the tables before closing out the sessions

In [22]:
## Drop the table before closing out the sessions

In [23]:
query = "drop table songs_played_in_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table songs_played_by_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

query = "drop table song_listeners"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)    

### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()